In [6]:
%load_ext autoreload
%autoreload 2
import torch
from torch.utils.data import TensorDataset, DataLoader
from torch import nn
from torch_scatter import scatter_add
import uproot
import awkward as ak

import sys
sys.path.append("..")
from dnn.ak_sample_loader import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
input = AkSampleLoader("/grid_mnt/data_cms_upgrade/cuisset/ticlRegression/PionSamples_v2/histo/histo_9376310_0.root")

In [8]:
from dnn.torch_dataset import *
ds = RegressionDataset(input, "feat-v2", 0, 500)

In [15]:
dl = makeDataLoader(makeValidationSample(input, "feat-v2", device="cpu"), batch_size=3)

In [16]:
list(dl)

[{'features': tensor([[3.7952e+00, 2.3598e+00, 4.0973e+02, 5.6846e+01, 1.6694e+00, 2.2043e-02,
           7.3238e+00, 1.7728e+00, 1.3252e+00, 5.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00, 9.5805e-01, 0.0000e+00, 2.8371e+00, 0.0000e+00],
          [2.9154e+00, 2.3868e+00, 4.0631e+02, 3.2293e+01, 1.2932e+00, 1.8214e-02,
           5.6809e+00, 1.1431e+00, 1.5931e-01, 4.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00, 1.9398e+00, 0.0000e+00, 9.7562e-01, 0.0000e+00],
          [1.4669e+02, 2.5863e+00, 3.4900e+02, 3.3902e+01, 1.4905e-02, 9.9249e-03,
           5.6555e+00, 3.3134e-01, 1.3534e+00, 1.6000e+01, 1.4669e+02, 0.0000e+00,
           0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [1.1240e+02, 2.5875e+00, 3.7661e+02, 2.6509e+02, 4.7283e+00, 3.4935e+00,
           1.5601e+01, 2.4170e+00, 4.9064e+00, 9.8000e+01, 3.2657e+01, 0.0000e+00,
           0.0000e+00, 7.6226e+01, 3.5164e+00, 0.0000e+00, 0.0000e+00],
          [3.3201e+00, 2.6343e+00, 4

In [17]:
list(dl)[0]

{'features': tensor([[3.7952e+00, 2.3598e+00, 4.0973e+02, 5.6846e+01, 1.6694e+00, 2.2043e-02,
          7.3238e+00, 1.7728e+00, 1.3252e+00, 5.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 9.5805e-01, 0.0000e+00, 2.8371e+00, 0.0000e+00],
         [2.9154e+00, 2.3868e+00, 4.0631e+02, 3.2293e+01, 1.2932e+00, 1.8214e-02,
          5.6809e+00, 1.1431e+00, 1.5931e-01, 4.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 1.9398e+00, 0.0000e+00, 9.7562e-01, 0.0000e+00],
         [1.4669e+02, 2.5863e+00, 3.4900e+02, 3.3902e+01, 1.4905e-02, 9.9249e-03,
          5.6555e+00, 3.3134e-01, 1.3534e+00, 1.6000e+01, 1.4669e+02, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [1.1240e+02, 2.5875e+00, 3.7661e+02, 2.6509e+02, 4.7283e+00, 3.4935e+00,
          1.5601e+01, 2.4170e+00, 4.9064e+00, 9.8000e+01, 3.2657e+01, 0.0000e+00,
          0.0000e+00, 7.6226e+01, 3.5164e+00, 0.0000e+00, 0.0000e+00],
         [3.3201e+00, 2.6343e+00, 4.3753e+02, 1.

In [36]:
dl = makeDataLoader(ds, batch_size=10, weighted=True)

In [37]:
batches = list(dl)
batches

[{'features': tensor([[3.7952e+00, 2.3598e+00, 4.0973e+02, 5.6846e+01, 1.6694e+00, 2.2043e-02,
           7.3238e+00, 1.7728e+00, 1.3252e+00, 5.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00, 9.5805e-01, 0.0000e+00, 2.8371e+00, 0.0000e+00],
          [2.9154e+00, 2.3868e+00, 4.0631e+02, 3.2293e+01, 1.2932e+00, 1.8214e-02,
           5.6809e+00, 1.1431e+00, 1.5931e-01, 4.0000e+00, 0.0000e+00, 0.0000e+00,
           0.0000e+00, 1.9398e+00, 0.0000e+00, 9.7562e-01, 0.0000e+00],
          [6.5073e+00, 2.1655e+00, 3.3313e+02, 3.9679e+01, 4.8126e-01, 2.4120e-01,
           6.0599e+00, 7.0714e-01, 1.7831e+00, 2.3000e+01, 6.5073e+00, 0.0000e+00,
           0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
          [7.3746e+01, 2.3424e+00, 3.9234e+02, 1.0280e+02, 1.8856e+00, 6.9775e-01,
           8.9226e+00, 3.3382e+00, 3.8239e+00, 1.7000e+01, 0.0000e+00, 0.0000e+00,
           0.0000e+00, 4.6038e+01, 2.0341e+01, 7.3668e+00, 0.0000e+00],
          [7.1663e+01, 2.3568e+00, 3

In [40]:
batch_i = 8
scatter_add(batches[batch_i]["features"][:, 0], batches[batch_i]["tracksterInEvent_idx"])

tensor([ 35.5057,  28.5874, 136.4660, 262.6303,  41.2875,  13.3857,  29.4358,
         12.2857, 139.6178, 376.2262])

In [41]:
batches[batch_i]["cp_energy"]

tensor([ 77.1409,  51.2673, 261.0424, 449.4850,  90.1634,  27.3100,  66.5357,
         81.7647, 475.9894, 473.4459])

In [32]:
scatter_add(batches[batch_i]["features"][:, 0], batches[batch_i]["tracksterInEvent_idx"])/batches[batch_i]["cp_energy"]

tensor([0.2244, 0.5176, 0.6950, 0.4424])

In [31]:
input.makeDataAk()[0:2]

<Array [[[3.8, 2.36, ..., 2.84, 0], ...], ...] type='2 * var * 10 * float32'>

In [11]:
np.cumsum(ak.num(input.tracksters_splitEndcaps.raw_energy))

<Array [2, 8, 12, 18, 19, 24, ..., 419, 421, 425, 431, 435] type='94 * int64'>

In [16]:
input.tracksters_splitEndcaps.raw_energy

<Array [[3.8, 2.92], ..., [227, 11, 5.73, 2.07]] type='94 * var * float32'>

In [18]:
ak.flatten(input.tracksters_splitEndcaps.raw_energy, axis=-1)[431:435]

<Array [227, 11, 5.73, 2.07] type='4 * float32'>

In [15]:
input.makeDataAk("feat-v2")

<Array [[[3.8, 2.36, ..., 2.84, 0], ...], ...] type='94 * var * 17 * float64'>

In [16]:
input.caloparticles_splitEndcaps

<Array [{time: 13.2, ...}, {...}, ..., {...}] type='20 * caloparticle[time:...'>

In [8]:
# enum :   enum CellType {
#     CE_E_120 = 0,
#     CE_E_200 = 1,
#     CE_E_300 = 2,
#     CE_H_120 = 3,
#     CE_H_200 = 4,
#     CE_H_300 = 5,
#     CE_H_SCINT = 6,
#     EnumSize = 7
#   };


In [18]:
input.energyPerCellType

<Array [[[0, 0.153, ..., 0.141, 0], ...], ...] type='20 * var * 7 * float32'>

## Version without clipping

In [19]:
true_energy = torch.tensor(input.caloparticles_splitEndcaps.regressed_energy)

In [51]:
tsInEvt_idx = torch.tensor(ak.flatten(ak.broadcast_arrays(ak.local_index(input.tracksters_splitEndcaps.raw_energy, axis=0), input.tracksters_splitEndcaps.raw_energy)[0]))
tsInEvt_idx.shape

torch.Size([44])

In [42]:
def akToTensor_noClip(ar:ak.Array):
    return torch.tensor(ak.flatten(ar))

In [54]:
data = torch.stack([akToTensor_noClip(tracksters_splitEndcaps.raw_energy), akToTensor_noClip(input.tracksters_splitEndcaps.barycenter_eta), akToTensor_noClip(input.tracksters_splitEndcaps.barycenter_z)] + [akToTensor_noClip(input.energyPerCellType[:, :, cellType_i]) for cellType_i in range(7)], dim=-1)

In [57]:
data.shape

torch.Size([44, 10])

In [60]:
class NetworkNoClip(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(data.shape[-1], 1, dtype=data.dtype),
        )

    def forward(self, x):
        return torch.squeeze(self.model(x))
net = NetworkNoClip()

In [61]:
net(data)

tensor([ 29.8255,  32.7520,  28.7608,  27.9655,  36.0864,  28.8798,  32.2802,
         25.1754,  33.4956,  26.7288,  28.3129,  25.7363,  27.1157,  27.5983,
         29.3331,  28.9366,  25.8991,  35.6060,  28.1522,  28.6858,  31.3700,
         28.5225,  32.2580, -32.6915, -33.0454, -32.0779, -32.0959, -35.1889,
        -30.3262, -29.3601, -29.6520, -47.7183, -41.3281, -29.7770, -31.6684,
        -29.0842, -32.9314, -31.8502, -34.3254, -29.0414, -29.1650, -36.3884,
        -34.7929, -34.5013], grad_fn=<SqueezeBackward0>)

In [65]:
scatter_add(net(data), tsInEvt_idx)

tensor([  62.5775,   28.7608,   64.0518,   61.1601,   58.6711,   55.0416,
          25.7363,   27.1157,   85.8680,  210.4936,  -32.6915,  -97.2192,
         -35.1889,  -89.3383,  -89.0464,  -90.5297,  -64.7816,  -34.3254,
         -58.2064, -105.6825], grad_fn=<ScatterAddBackward0>)

In [66]:
loss_fct = nn.MSELoss()
loss = loss_fct(scatter_add(net(data), tsInEvt_idx), true_energy)
loss

tensor(7357.2197, grad_fn=<MseLossBackward0>)

In [69]:
scatter_add(data[:, 0], tsInEvt_idx)/true_energy

tensor([0.3364, 0.1629, 0.3169, 0.4494, 0.3576, 0.3785, 0.6750, 0.4717, 0.4358,
        0.4709, 0.8010, 0.6355, 0.8678, 0.3077, 0.6035, 0.3467, 0.5526, 0.7568,
        0.0697, 0.4638])